<h1> Data Exploration and Train-Test Split</h1>

<h2>**Part 1: Data Exploration and Pre-processing**</h2>

<h3> Step 1: Import Kaggle and Connect to Kaggle via token </h3>

In [ ]:
! pip install -q kaggle

In [ ]:
from google.colab import userdata
import os

os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')
os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')

In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.4/256.4 kB 9.1 MB/s eta 0:00:00
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.7.4.5
    Uninstalling kaggle-1.7.4.5:
      Successfully uninstalled kaggle-1.7.4.5


<h3> Step 2: Download dataset via kaggle</h3>

In [ ]:
!kaggle datasets download -d "notsalmankhan/amazon-esci-shopping-queries"



Dataset URL: https://www.kaggle.com/datasets/notsalmankhan/amazon-esci-shopping-queries
License(s): apache-2.0
 99% 939M/948M [00:13<00:00, 133MB/s]
100% 948M/948M [00:14<00:00, 70.9MB/s]


In [ ]:
! unzip "amazon-esci-shopping-queries.zip"

Archive:  amazon-esci-shopping-queries.zip
  inflating: shopping_queries_dataset_examples.parquet  
  inflating: shopping_queries_dataset_products.parquet  
  inflating: shopping_queries_dataset_sources.csv  


<h3> Step 3: Mount drive to read from/ write to it </h3>

In [ ]:
import pandas as pd
import gc
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<h3> Step 4: Read unzipped dataset files into dataframes </h3>

In [ ]:
df_examples = pd.read_parquet('shopping_queries_dataset_examples.parquet')
df_products = pd.read_parquet('shopping_queries_dataset_products.parquet')
df_sources = pd.read_csv("shopping_queries_dataset_sources.csv")

In [ ]:
df_examples.head()

,example_id,query,query_id,product_id,product_locale,esci_label,small_version,large_version,split
0,0,revent 80 cfm,0,B000MOO21W,us,I,0,1,train
1,1,revent 80 cfm,0,B07X3Y6B1V,us,E,0,1,train
2,2,revent 80 cfm,0,B07WDM7MQQ,us,E,0,1,train
3,3,revent 80 cfm,0,B07RH6Z8KW,us,E,0,1,train
4,4,revent 80 cfm,0,B07QJ7WYFQ,us,E,0,1,train


In [ ]:
df_products.head()

,product_id,product_title,product_description,product_bullet_point,product_brand,product_color,product_locale
0,B079VKKJN7,"11 Degrees de los Hombres Playera con Logo, Ne...",Esta playera con el logo de la marca Carrier d...,11 Degrees Negro Playera con logo\nA estrenar ...,11 Degrees,Negro,es
1,B079Y9VRKS,Camiseta Eleven Degrees Core TS White (M),None,None,11 Degrees,Blanco,es
2,B07DP4LM9H,11 Degrees de los Hombres Core Pull Over Hoodi...,La sudadera con capucha Core Pull Over de 11 G...,11 Degrees Azul Core Pull Over Hoodie\nA estre...,11 Degrees,Azul,es
3,B07G37B9HP,11 Degrees Poli Panel Track Pant XL Black,None,None,11 Degrees,None,es
4,B07LCTGDHY,11 Degrees Gorra Trucker Negro OSFA (Talla úni...,None,None,11 Degrees,Negro (,es


In [ ]:
df_sources.head()

,query_id,source
0,0,other
1,1,negations
2,2,negations
3,3,negations
4,4,behavioral


In [ ]:
len(df_examples)

2621288

<h3> Step 5: Merge dataframes for query-product relevance and product details based on product id and locale </h3>

In [ ]:
df_examples_products = pd.merge(
    df_examples,
    df_products,
    how='left',
    left_on=['product_locale','product_id'],
    right_on=['product_locale', 'product_id']
)

In [ ]:
del df_examples
del df_products
del df_sources
gc.collect()

0

<h3> Step 6: Retain data marked as small_version </h3>

In [ ]:
df_task_1 = df_examples_products[df_examples_products["small_version"] == 1]

In [ ]:
len(df_task_1)

1118011

<h3> Step 7: Remove data corresponding to locales other than us </h3>

In [ ]:
df_task_1['product_locale'].unique()

array(['us', 'es', 'jp'], dtype=object)

In [ ]:
df_task_1=df_task_1[df_task_1['product_locale']=='us']

In [ ]:
df_task_1['product_locale'].unique()

array(['us'], dtype=object)

In [ ]:
len(df_task_1)

601354

In [ ]:
df_task_1.head()

,example_id,query,query_id,product_id,product_locale,esci_label,small_version,large_version,split,product_title,product_description,product_bullet_point,product_brand,product_color
16,16,!awnmower tires without rims,1,B075SCHMPY,us,I,1,1,train,"RamPro 10"" All Purpose Utility Air Tires/Wheel...","<b>About The Ram-Pro All Purpose Utility 10"" A...",✓ The Ram-Pro Ten Inch ready to install Air Ti...,RamPro,10 Inch
17,17,!awnmower tires without rims,1,B08L3B9B9P,us,E,1,1,train,MaxAuto 2-Pack 13x5.00-6 2PLY Turf Mower Tract...,MaxAuto 2-Pack 13x5.00-6 2PLY Turf Mower Tract...,Please check your existing tire Sidewall for t...,MaxAuto,None
18,18,!awnmower tires without rims,1,B082K7V2GZ,us,I,1,1,train,NEIKO 20601A 14.5 inch Steel Tire Spoon Lever ...,None,[QUALITY]: Hardened Steel-Iron construction wi...,Neiko,None
19,19,!awnmower tires without rims,1,B07P4CF3DP,us,S,1,1,train,2PK 13x5.00-6 13x5.00x6 13x5x6 13x5-6 2PLY Tur...,"Tire Size: 13 x 5.00 - 6 Axle: 3/4"" inside dia...",None,Russo,None
20,20,!awnmower tires without rims,1,B07C1WZG12,us,E,1,1,train,(Set of 2) 15x6.00-6 Husqvarna/Poulan Tire Whe...,No fuss. Just take off your old assembly and r...,Tire size:15x6.00-6 Ply: 4 Tubeless\n6x4.5 Whe...,Antego Tire & Wheel,Husqvarna Silver


<h3> Step 8 (Checkpoint): Write preprocessed dataframe to drive in csv format </h3>

In [ ]:
file_name = 'df_task_1.csv'
save_path = os.path.join('/content/drive/MyDrive/', file_name)

df_task_1.to_csv(save_path, index=False)

<h3> Step 9: Drop columns 'example_id', 'query_id', 'product_locale', 'small_version', 'large_version', 'product_id' and all records where there are missing values </h3>

In [ ]:
df_task_1=pd.read_csv('/content/drive/MyDrive/df_task_1.csv', index_col=False)

In [ ]:
columns_to_drop = ['example_id', 'query_id', 'product_locale', 'small_version', 'large_version' , 'product_id']
df_task_1 = df_task_1.drop(columns_to_drop, axis=1)

In [ ]:
df_task_1_wo_null = df_task_1.dropna(axis=0)

<h2> **Part 2: Train-Test Split and further processing to make data suitable to feed as input to our training code** </h2>

<h3> Step 10: Split data into train: and test dataframes based on the value of "split" column </h3>

In [ ]:
df_task_1_train = df_task_1_wo_null[df_task_1_wo_null["split"] == "train"]
df_task_1_test = df_task_1_wo_null[df_task_1_wo_null["split"] == "test"]

In [ ]:
df_task_1_train = df_task_1_train.drop('split', axis=1)
df_task_1_test = df_task_1_test.drop('split', axis=1)

In [ ]:
del df_task_1_wo_null
gc.collect()

16

<h3> Step 11: Move half data from test to train dataframe to achieve train-test split of approximately 85%-15% </h3>

In [ ]:
half_records=len(df_task_1_test)//2

In [ ]:
half_df_test = df_task_1_test.iloc[:half_records]

In [ ]:
move_df_test = df_task_1_test.iloc[half_records:]

In [ ]:
df_task_1_train = pd.concat([df_task_1_train, move_df_test], ignore_index=True)

In [ ]:
df_task_1_test=half_df_test

In [ ]:
del move_df_test
del half_df_test
gc.collect()

0

<h3> Step 12: Add a column called Query Item Pair which concatenates query and product details. Drop the other query and product detail related columns </h3>

In [ ]:
df_task_1_train['Query Item Pair']= 'Query: '+df_task_1_train['query'].astype(str)+' Product Title: '+df_task_1_train['product_title'].astype(str)+' Product Description: '+df_task_1_train['product_description'].astype(str)+' Product Bullet Point: '+df_task_1_train['product_bullet_point'].astype(str)+' Product Brand: '+df_task_1_train['product_brand'].astype(str)+' Product Color: '+df_task_1_train['product_color'].astype(str)
df_task_1_train = df_task_1_train.drop(['query','product_title','product_description','product_bullet_point','product_brand', 'product_color'],axis = 1)

In [ ]:
df_task_1_train.head()

,esci_label,Query Item Pair
0,I,Query: !awnmower tires without rims Product Ti...
1,E,Query: !awnmower tires without rims Product Ti...
2,E,Query: !awnmower tires without rims Product Ti...
3,I,Query: !awnmower tires without rims Product Ti...
4,E,Query: # 10 self-seal envelopes without window...


In [ ]:
max_words = df_task_1_train['Query Item Pair'].str.split().str.len().max()
print(f"The maximum number of words in the column is: {max_words}")


The maximum number of words in the column is: 988


In [ ]:
df_task_1_test['Query Item Pair']= 'Query: '+df_task_1_test['query'].astype(str)+' Product Title: '+df_task_1_test['product_title'].astype(str)+' Product Description: '+df_task_1_test['product_description'].astype(str)+' Product Bullet Point: '+df_task_1_test['product_bullet_point'].astype(str)+' Product Brand: '+df_task_1_test['product_brand'].astype(str)+' Product Color: '+df_task_1_test['product_color'].astype(str)
df_task_1_test = df_task_1_test.drop(['query','product_title','product_description','product_bullet_point','product_brand', 'product_color'],axis = 1)

In [ ]:
df_task_1_test.head()

,esci_label,Query Item Pair
16,I,Query: !qscreen fence without holes Product Ti...
21,I,Query: !qscreen fence without holes Product Ti...
24,I,Query: !qscreen fence without holes Product Ti...
25,E,Query: !qscreen fence without holes Product Ti...
26,E,Query: !qscreen fence without holes Product Ti...


<h3> Step 13: Map E,S,C,I values to column soft_target which would contain values 1, 0.5, 0 where 1 indicates high relevance and 0 indicates low relevance. Drop the esci_label column. </h3>

In [ ]:
conditions = [
    (df_task_1_train['esci_label'] == 'E'),
    (df_task_1_train['esci_label'] == 'S'),
    (df_task_1_train['esci_label'] == 'C'),
    (df_task_1_train['esci_label'] == 'I')
]
choices = [1, 0.5, 0, 0]

df_task_1_train['soft_target'] = np.select(conditions, choices, default=0)

In [ ]:
df_task_1_train.head()

,esci_label,Query Item Pair,soft_target
0,I,Query: !awnmower tires without rims Product Ti...,0.0
1,E,Query: !awnmower tires without rims Product Ti...,1.0
2,E,Query: !awnmower tires without rims Product Ti...,1.0
3,I,Query: !awnmower tires without rims Product Ti...,0.0
4,E,Query: # 10 self-seal envelopes without window...,1.0


In [ ]:
df_task_1_train=df_task_1_train.drop('esci_label', axis=1)

In [ ]:
df_task_1_train.head()

,Query Item Pair,soft_target
0,Query: !awnmower tires without rims Product Ti...,0.0
1,Query: !awnmower tires without rims Product Ti...,1.0
2,Query: !awnmower tires without rims Product Ti...,1.0
3,Query: !awnmower tires without rims Product Ti...,0.0
4,Query: # 10 self-seal envelopes without window...,1.0


In [ ]:
conditions = [
    (df_task_1_test['esci_label'] == 'E'),
    (df_task_1_test['esci_label'] == 'S'),
    (df_task_1_test['esci_label'] == 'C'),
    (df_task_1_test['esci_label'] == 'I')
]
choices = [1, 0.5, 0, 0]

df_task_1_test['soft_target'] = np.select(conditions, choices, default=0)

In [ ]:
df_task_1_test.head()

,esci_label,Query Item Pair,soft_target
16,I,Query: !qscreen fence without holes Product Ti...,0.0
21,I,Query: !qscreen fence without holes Product Ti...,0.0
24,I,Query: !qscreen fence without holes Product Ti...,0.0
25,E,Query: !qscreen fence without holes Product Ti...,1.0
26,E,Query: !qscreen fence without holes Product Ti...,1.0


In [ ]:
df_task_1_test=df_task_1_test.drop('esci_label', axis=1)

In [ ]:
df_task_1_test.head()

,Query Item Pair,soft_target
16,Query: !qscreen fence without holes Product Ti...,0.0
21,Query: !qscreen fence without holes Product Ti...,0.0
24,Query: !qscreen fence without holes Product Ti...,0.0
25,Query: !qscreen fence without holes Product Ti...,1.0
26,Query: !qscreen fence without holes Product Ti...,1.0


In [ ]:
print(len(df_task_1_train))
print(len(df_task_1_test))

175329
31020


<h3> Step 14 (Checkpoint): Write train and test dataframe to drive in csv format </h3>

In [ ]:
file_name = 'df_task_1_train.csv'
save_path = os.path.join('/content/drive/MyDrive/', file_name)

df_task_1_train.to_csv(save_path, index=False)

In [ ]:
file_name = 'df_task_1_test.csv'
save_path = os.path.join('/content/drive/MyDrive/', file_name)

df_task_1_test.to_csv(save_path, index=False)